# Import and sandbox

In [1]:
import numpy as np
import pandas as pd
import torch 
from tqdm import tqdm
import matplotlib.pyplot as plt


# check n_wells 

## Imports

In [2]:
df_phenom = pd.read_parquet('/projects/synsight/data/jump_embeddings/wells_embeddings/openphenom/metadata_openphenom.parquet')

In [3]:
df_phenom.drop(columns=['Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChI'], inplace=True)

In [4]:
df_phenom = df_phenom.drop_duplicates()

In [5]:
import pandas as pd

# Charger vos données
# df_phenom = pd.read_csv("votre_fichier.csv") # Si nécessaire

# Calculer le nombre maximum de lignes que l'on peut équilibrer
min_samples_per_source = df_phenom.groupby('Metadata_Source').size().min()

# Échantillonner de façon équilibrée entre les sources et diversifiée sur les batches
balanced_sample = df_phenom.groupby('Metadata_Source').apply(
    lambda group: group.groupby('Metadata_Batch').apply(
        lambda batch: batch.sample(n=min(len(batch), min_samples_per_source // len(group['Metadata_Batch'].unique())))
    ).reset_index(drop=True)
).reset_index(drop=True)



/tmp/ipykernel_596226/3905330124.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lambda group: group.groupby('Metadata_Batch').apply(
/tmp/ipykernel_596226/3905330124.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lambda group: group.groupby('Metadata_Batch').apply(
/tmp/ipykernel_596226/3905330124.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior i

In [6]:
balanced_sample


,Metadata_Source,Metadata_Batch,Metadata_Plate
0,source_1,Batch1_20221004,UL001655
1,source_1,Batch1_20221004,UL001643
2,source_1,Batch1_20221004,UL001651
3,source_1,Batch1_20221004,UL001641
4,source_1,Batch1_20221004,UL001653
...,...,...,...
383,source_9,20211102-Run15,GR00004396
384,source_9,20211103-Run16,GR00004413
385,source_9,20211103-Run16,GR00004421
386,source_9,20211103-Run16,GR00004419


In [21]:
import pandas as pd

# Charger vos données
# df_phenom = pd.read_csv("votre_fichier.csv")  # Charger votre DataFrame si nécessaire

# Calculer le nombre maximum de lignes que l'on peut équilibrer par source
min_samples_per_source = df_phenom.groupby('Metadata_Source').size().min()

# Fonction pour échantillonner de manière diversifiée
def balanced_sampling(group, n_samples):
    unique_batches = group['Metadata_Batch'].unique()
    n_batches = len(unique_batches)
    batch_sample_size = max(1, n_samples // n_batches)
    
    sampled = group.groupby('Metadata_Batch').apply(
        lambda batch: batch.sample(min(len(batch), batch_sample_size))
    ).reset_index(drop=True)
    return sampled.head(n_samples)  # Limiter au total nécessaire

# Appliquer l'échantillonnage équilibré
balanced_sample = df_phenom.groupby('Metadata_Source').apply(
    lambda group: balanced_sampling(group, min_samples_per_source)
).reset_index(drop=True)

# Supprimer les doublons (pour vérifier s'il y en avait)
balanced_sample = balanced_sample.drop_duplicates()

# Résultat
print(balanced_sample.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Metadata_Source  388 non-null    object
 1   Metadata_Batch   388 non-null    object
 2   Metadata_Plate   388 non-null    object
dtypes: object(3)
memory usage: 9.2+ KB
None


/tmp/ipykernel_320105/3054499886.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled = group.groupby('Metadata_Batch').apply(
/tmp/ipykernel_320105/3054499886.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled = group.groupby('Metadata_Batch').apply(
/tmp/ipykernel_320105/3054499886.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprec

In [22]:
balanced_sample

,Metadata_Source,Metadata_Batch,Metadata_Plate
0,source_1,Batch1_20221004,UL001653
1,source_1,Batch1_20221004,UL001643
2,source_1,Batch1_20221004,UL001645
3,source_1,Batch1_20221004,UL001651
4,source_1,Batch1_20221004,UL001655
...,...,...,...
383,source_9,20211102-Run15,GR00004391
384,source_9,20211103-Run16,GR00004414
385,source_9,20211103-Run16,GR00004411
386,source_9,20211103-Run16,GR00004418


In [24]:
import pandas as pd

# Calculer le nombre maximum de plaques échantillonnées uniformément
min_samples_per_source = df_phenom.groupby('Metadata_Source').size().min()

# Échantillonner un nombre fixe de plaques par source
balanced_sample = df_phenom.groupby('Metadata_Source').apply(
    lambda group: group.sample(n=min_samples_per_source, random_state=42)  # Fixer un seed pour reproductibilité
).reset_index(drop=True)

# Résultat final
print(balanced_sample['Metadata_Source'].value_counts())


Metadata_Source
source_1     51
source_10    51
source_11    51
source_2     51
source_3     51
source_5     51
source_6     51
source_8     51
source_9     51
Name: count, dtype: int64


/tmp/ipykernel_320105/3147042607.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_sample = df_phenom.groupby('Metadata_Source').apply(


In [7]:
balanced_sample['Metadata_Source'].value_counts()

Metadata_Source
source_10    51
source_11    50
source_8     48
source_1     45
source_9     44
source_3     39
source_2     39
source_6     39
source_5     33
Name: count, dtype: int64

In [8]:
balanced_sample

,Metadata_Source,Metadata_Batch,Metadata_Plate
0,source_1,Batch1_20221004,UL001655
1,source_1,Batch1_20221004,UL001643
2,source_1,Batch1_20221004,UL001651
3,source_1,Batch1_20221004,UL001641
4,source_1,Batch1_20221004,UL001653
...,...,...,...
383,source_9,20211102-Run15,GR00004396
384,source_9,20211103-Run16,GR00004413
385,source_9,20211103-Run16,GR00004421
386,source_9,20211103-Run16,GR00004419


In [10]:
balanced_sample.to_json("balanced_plates.json", orient="records")


In [31]:
imported_plates

['UL000087',
 'UL001653',
 'UL001781',
 'UL001713',
 'UL001649',
 'UL001793',
 'UL000081',
 'UL001645',
 'UL000093',
 'UL001799',
 'UL001795',
 'UL001769',
 'UL001647',
 'UL001791',
 'UL001785',
 'UL001789',
 'UL001719',
 'UL000579',
 'UL000089',
 'UL001673',
 'UL001765',
 'UL000095',
 'UL000583',
 'UL000083',
 'UL000091',
 'UL001675',
 'UL000599',
 'UL000595',
 'UL001783',
 'UL000593',
 'UL001771',
 'UL001717',
 'UL001677',
 'UL001651',
 'UL000577',
 'UL000585',
 'UL001669',
 'UL001641',
 'UL000597',
 'UL000587',
 'UL001665',
 'UL000581',
 'UL001643',
 'UL001773',
 'UL001797',
 'UL001655',
 'UL001679',
 'UL001667',
 'UL001787',
 'UL000097',
 'UL001671',
 'Dest210803-160838',
 'Dest210707-094057',
 'Dest210823-175531',
 'Dest210726-161801',
 'Dest210823-180215',
 'Dest210823-173115',
 'Dest210810-174829',
 'Dest210705-143133',
 'Dest210621-134650',
 'Dest210803-154444',
 'Dest210615-151512',
 'Dest210621-132542',
 'Dest210621-133202',
 'Dest210705-143821',
 'Dest210727-155549',
 'Dest2

In [9]:
balanced_sample['Metadata_Batch'].value_counts()

Metadata_Batch
J3                                        12
J2                                        12
J1                                        12
J4                                        12
Batch5                                    10
                                          ..
JUMPCPE-20210623-Run02_20210624_225846     1
JUMPCPE-20211014-Run36_20211014_223431     1
JUMPCPE-20211007-Run35_20211007_235529     1
JUMPCPE-20211001-Run34_20211003_121618     1
JUMPCPE-20211001-Run33_20211001_152017     1
Name: count, Length: 115, dtype: int64